<div dir="rtl" align="right">

# شروع کار با LangChain و Groq
در این بخش، با نحوه بارگذاری کلید Groq و آماده‌سازی محیط برای کار با LangChain و مدل‌های Groq آشنا می‌شویم.

</div>

<div dir="rtl" align="right">

### بارگذاری کلید Groq و آماده‌سازی محیط
در این سلول، کلید Groq را از فایل `.env` بارگذاری می‌کنیم تا بتوانیم به مدل‌های زبانی Groq دسترسی داشته باشیم.

</div>

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

<div dir="rtl" align="right">

### ساخت مدل زبانی Groq
در این سلول، یک مدل زبانی Groq را با تنظیمات دلخواه مقداردهی می‌کنیم.

</div>

In [ ]:
from langchain_groq import ChatGroq

#Mitoonid az model llama-3.3-70b-versatile ham estefade konid

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.6)
response = llm.invoke("Generate a list of 10 names for a new product.")
name_list = response.content.split("\n")
print(name_list)

['Here are 10 name suggestions for a new product:', '', '1. **Aurora**: A name that evokes the idea of a new dawn and suggests innovation and progress.', '2. **Lumina**: A name that means "light" in Latin, implying illumination and understanding.', "3. **Pulse**: A name that conveys energy and vitality, suggesting a product that's dynamic and engaging.", '4. **Nexa**: A name that implies connection and integration, suggesting a product that brings people or things together.', "5. **Flux**: A name that suggests change and adaptability, implying a product that's flexible and responsive.", "6. **Spectra**: A name that implies a range of possibilities and suggests a product that's versatile and multifaceted.", "7. **Apex**: A name that conveys excellence and achievement, suggesting a product that's top-notch and reliable.", '8. **Nova**: A name that means "new" in Latin, implying a product that\'s innovative and groundbreaking.', '9. **Catalyst**: A name that suggests a product that sparks

<div dir="rtl" align="right">

### آشنایی با PromptTemplate
در این سلول، با مفهوم قالب‌های پرامپت (PromptTemplate) آشنا می‌شویم و یک نمونه ساده با مدل Groq می‌سازیم.

</div>

In [11]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="Write a short message in {language} and in a {tone} tone: {message}",
    input_variables=["tone", "message", "language"],
)

filled_prompt = prompt.format(tone="friendly", language="Persian", message="Welcome to LangChain!")
print(filled_prompt)

Write a short message in Persian and in a friendly tone: Welcome to LangChain!


<div dir="rtl" align="right">

#### استفاده از قالب پرامپت برای تولید پیام با مدل Groq
</div>

In [12]:
response = llm.invoke(filled_prompt)
print(response.content)

خوش آمدید به LangChain


<div dir="rtl" align="right">

## گرفتن خروجی مدل Groq با استفاده از قالب پرامپت
در این سلول، پیام ساخته‌شده را به مدل Groq می‌دهیم و خروجی مدل را مشاهده می‌کنیم.

</div>

In [13]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Title prompt template
prompt_title = PromptTemplate(
    template="Generate a catchy tutorial title in {language} and in a {tone} tone for: {message}",
    input_variables=["tone", "message", "language"],
)

# Description prompt template
prompt_desc = PromptTemplate(
    template="Write a short description for a tutorial titled '{title}' in {language} and in a {tone} tone. The topic is: {message}",
    input_variables=["tone", "message", "language", "title"],
)

# Output parser
output_parser = StrOutputParser()

# Title chain
title_chain = prompt_title | llm | output_parser

# Description chain (takes the output of title_chain)
desc_chain = (
    RunnablePassthrough.assign(title=title_chain)
    | prompt_desc
    | llm
    | output_parser
)

# Example inputs
inputs = {"tone": "friendly", "language": "English", "message": "How to use LangChain for beginners"}

title = title_chain.invoke(inputs)
description = desc_chain.invoke(inputs)

print("Title:", title)
print("Description:", description)

Title: Here are a few options for a catchy tutorial title in English and a friendly tone:

1. **"Link Up: A Beginner's Guide to LangChain"**
2. **"Chatbot Magic with LangChain: A Step-by-Step Tutorial"**
3. **"LangChain for Newbies: Build Your First Chatbot in Minutes"**
4. **"Unlock LangChain's Power: A Beginner's Journey to AI Mastery"**
5. **"LangChain 101: From Zero to Hero in Chatbot Development"**
6. **"Get Started with LangChain: The Ultimate Beginner's Tutorial"**
7. **"LangChain Made Easy: A Beginner's Guide to Building Intelligent Chatbots"**
8. **"LangChain for Beginners: A Fun and Interactive Learning Experience"**
9. **"LangChain in a Nutshell: A Beginner's Guide to Chatbot Development"**
10. **"LangChain Unleashed: A Beginner's Guide to Building AI-Powered Chatbots"**

Choose the one that resonates with you the most, or feel free to modify any of these titles to fit your style!
Description: **"Chat Like a Pro: A Beginner's Guide to LangChain"**

Get ready to unlock the se

<div dir="rtl" align="right">

## پیاده‌سازی زنجیره تولید عنوان و توضیح آموزش با مدل Groq
در این سلول، دو زنجیره ساده برای تولید عنوان و توضیح آموزش با مدل Groq پیاده‌سازی می‌کنیم و نتیجه را نمایش می‌دهیم.

</div>

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Title prompt template
prompt_title = PromptTemplate(
    template="Generate a catchy tutorial title in {language} and in a {tone} tone for: {message}",
    input_variables=["tone", "message", "language"],
)

# Description prompt template
prompt_desc = PromptTemplate(
    template="Write a short description for a tutorial titled '{title}' in {language} and in a {tone} tone. The topic is: {message}",
    input_variables=["tone", "message", "language", "title"],
)

# Output parser
output_parser = StrOutputParser()

# Title chain
title_chain = prompt_title | llm | output_parser

# Description chain (takes the output of title_chain)
desc_chain = (
    RunnablePassthrough.assign(title=title_chain)
    | prompt_desc
    | llm
    | output_parser
)

# Example inputs
inputs = {"tone": "friendly", "language": "Persian", "message": "How to use LangChain for beginners"}

title = title_chain.invoke(inputs)
description = desc_chain.invoke(inputs)

print("Title:", title)
print("Description:", description)

Title: کد نویسی آسان با LangChain برای مبتدیان: چگونه با استفاده از LangChain یک سیستم یادگیری ماشین قدرتمند بسازید

(Translation: Easy Coding with LangChain for Beginners: How to Create a Powerful Machine Learning System using LangChain)

Alternatively:

راهنمای کامل LangChain برای مبتدیان: از مقدمات تا ساخت سیستم های یادگیری ماشین پیشرفته

(Translation: Complete Guide to LangChain for Beginners: From Basics to Building Advanced Machine Learning Systems)

Alternatively:

 LangChain در 10 دقیقه: یک مقدمه ی جامع برای مبتدیان

(Translation: LangChain in 10 Minutes: A Comprehensive Introduction for Beginners)
Description: **عنوان:** LangChain را در 5 دقیقه بیاموزید: چگونه یک برنامه‌نویس مبتدی LangChain را به کار ببندد!

**توضیحات:** در این دوره آموزشی جامع و آسان، به شما یاد خواهیم داد که چگونه LangChain را به کار ببندید و از آن برای پروژه‌های خود استفاده کنید. ما در این دوره به مباحث مقدماتی LangChain می‌پردازیم و شما را از طریق مراحل مختلف یک پروژه آشنا می‌کنیم. با ما همراه باشید و در پ

<div dir="rtl" align="right">

الان می‌تونید با تغییر «tone»، «language» یا «message» در دیکشنری «inputs» آزمایش کنید تا ببینید عنوان و توضیحات چجوری تغییر می‌کنند. این نشون میده که چجوری زنجیره‌ها به ما امکان می‌دهند گردش‌های کاری پیچیده‌تر و ماژولارتری را در LangChain با مدل Groq بسازیم.

</div>